In [12]:
# Modelo de recomendación de juegos

# El siguiente modelo de recomendación propone una lista de 5 juegos ingresando el nombre de un juego. Este modelo tiene una relación ítem-ítem, ya que toma un juego y en base a que tan similar es ese juego con el resto de los juegos se recomiendan similares.

In [13]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

import pyarrow as pa
import pyarrow.parquet as pq

In [14]:
df = pd.read_csv("C:\\Users\\cquir\\OneDrive\\Escritorio\\Data Science SH\\Proyecto Individual 1\\bases de datos\\df_recomendacion")
df

,user_id,item_name,rating
0,76561197970982479,Killing Floor,3
1,evcentric,Risk of Rain,5
2,doctr,The Wolf Among Us,5
3,maplemage,Dark Souls: Prepare to Die Edition,3
4,Wackky,LEGO® MARVEL Super Heroes,1
...,...,...,...
44200,76561198107177722,BattleBlock Theater,5
44201,kushikushigani,LEGO® Worlds,3
44202,76561198111410893,Unturned,3
44203,zaza147,Fistful of Frags,5


In [4]:
# Creamos un df que contiene los 'user_id' como indices, los juegos ('item_name') como columnas y como valores los ´rating'.
piv = df.pivot_table(index=['user_id'], columns=['item_name'], values='rating')
piv

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,140,16bit Trader,...,inMomentum,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theHunter,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ace--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ionex--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2SV-vuLB-Kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Azsael-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuzuga2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zv_odd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zvanik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Normalizamos los valores del df piv. Esta normalización consiste en una transformación de los datos para que los mismos estén en un rango común (ya que muchas veces los datos están en escalas u ordenes de magnitud muy diferentes), y así no haya predominancia o preponderancia de un tipo de dato sobre el otro. Así, se estandarizan y se reduce el impacto de las diferencias de escala y la magnitud de atributos de los mismos.
# A los usuarios que solo han dado una calificación o han calificado todos los juegos de la misma manera serán eliminados durante este proceso de normalización. Esto se debe a que estos usuarios no aportan información útil para el modelo de recomendación si todas sus calificaciones son iguales o si solo tienen una calificación.

# Normalización del dataframe 'piv'
piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

# Se borran las columnas que contienen solo cero o no tienen rating, se rellenan los vacios con 0 y se hace la transpues
piv_norm = piv_norm.fillna(0)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:, (piv_norm!=0).any(axis=0)]   #La finalidad de este script es eliminar las columnas que estan completamente llenas de ceros
piv_norm

user_id,--000--,-Beave-,-I_AM_EPIC-,-SEVEN-,-Thyme-,-kainey9777,00000000000000000001227,00690069006900,03092002,04061993,...,zombi_anon,zomgCoBfAce,zoom-the-flash,zoozles,zourock,zsharoarkbr,zuzuga2003,zvanik,zwanzigdrei,zzoptimuszz
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001 Spikes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
resident evil 4 / biohazard 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# A estos datos se les convierte a un formato de matriz dispersa para reducir la memoria de uso y mejorar la eficiencia en el manejo de grandes conjuntos de datos. Esto, ya que la matriz dispersa almacena solo los valores distintos de cero junto con su ubicación en la matriz, en lugar de almacenar todos los valores de la matriz, incluso los ceros.
piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
piv_sparse

<2813x6963 sparse matrix of type '<class 'numpy.float64'>'
	with 25039 stored elements in Compressed Sparse Row format>

In [7]:
# Creamos una matriz de similitud utilizando la similitud del coseno para medir la similitud entre juegos (item_similarity). Esta similitud es usada para determinar cuán similares son dos conjuntos de datos o elementos.
item_similarity = cosine_similarity(piv_sparse)

In [8]:
# Para estructurar y organizar los resultados de manera más accesible y comprensible se insertan las matrices anteriores en un Dataframe.
item_sim_df = pd.DataFrame(item_similarity, index = piv_norm.index, columns = piv_norm.index)

In [9]:
# Función de recomendación según un juego

# Ahora, conociendo la relación entre los distintos juegos, se puede proponer una función que realice una recomendación de 5 juegos en función de un juego dado, teniendo en cuenta los valores mas altos de similitud del coseno. Esta función toma un nombre de un juego como entrada, luego ordena la columna correspondiente a ese juego en la matriz de similitud entre elementos (item_sim_df) de manera descendente, de modo que los juegos más similares aparezcan en la parte superior. Posteriormente selecciona los 5 juegos más similares (excluyendo el propio juego que se pasó como entrada), itera a través de estos juegos similares y, finalmente, imprime una lista de juegos similares al juego especificado.

def top_game(game):
    count = 1
    print('juegos similares a {} incluyen:\n'.format(game))
    for item in item_sim_df.sort_values(by=game, ascending=False).index[1:6]:
        print('No. {}: {}'.format(count,item))
        count += 1

In [10]:
top_game('Unreal Gold')

juegos similares a Unreal Gold incluyen:

No. 1: Killing Floor
No. 2: ! That Bastard Is Trying To Steal Our Gold !
No. 3: Retrovirus
No. 4: Resident Evil Revelations / Biohazard Revelations
No. 5: Resident Evil Revelations 2 / Biohazard Revelations 2


In [17]:
pq.write_table(pa.Table.from_pandas(piv_norm), 'C:\\Users\\cquir\\OneDrive\\Escritorio\\Data Science SH\\Proyecto Individual 1\\bases de datos\\piv_norm.parquet')
pq.write_table(pa.Table.from_pandas(item_sim_df), 'C:\\Users\\cquir\\OneDrive\\Escritorio\\Data Science SH\\Proyecto Individual 1\\bases de datos\\item_sim_df.parquet')